In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.
/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tampm/miniconda3/env

In [3]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.2
# config.gpu_options.visible_device_list = "0" #only the gpu 0 is allowed
set_session(tf.Session(config=config))

In [4]:
from __future__ import print_function
from __future__ import absolute_import

import warnings
import numpy as np

from keras.preprocessing import image
from keras.applications import Xception
# from keras.applications import Xception, VGG16

from keras.models import Model
import keras
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Conv2D
from keras.layers import SeparableConv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras import backend as K

from keras.layers import Layer
from keras import regularizers

In [5]:
# https://github.com/4uiiurz1/keras-arcface
class ArcFace(Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(theta + self.m)
        # sin = tf.sqrt(1 - logits**2)
        # cos_m = tf.cos(logits)
        # sin_m = tf.sin(logits)
        # target_logits = logits * cos_m - sin * sin_m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)


class SphereFace(Layer):
    def __init__(self, n_classes=10, s=30.0, m=1.35, regularizer=None, **kwargs):
        super(SphereFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(SphereFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(self.m * theta)
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)


class CosFace(Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.35, regularizer=None, **kwargs):
        super(CosFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(CosFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        target_logits = logits - self.m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)

In [6]:
input = Input(shape=(128, 128, 3))
label = Input(shape=(2,))
x = Xception(include_top=False, weights="imagenet", input_shape=(128, 128, 3))(input)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(2,activation = 'softmax',kernel_initializer='he_normal')(x)
output = ArcFace(n_classes=2)([x, label])
model = Model([input, label], output)

    

In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
xception (Model)                (None, 4, 4, 2048)   20861480    input_1[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 32768)        0           xception[1][0]                   
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 2)            65538       flatten_1[0][0]                  
__________________________________________________________________________________________________
input_2 (I

In [8]:
model.compile(optimizer = "adam", loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [9]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 8

In [24]:
import time

In [25]:
time.sleep(0.0001)

In [10]:
dataGenerator = ImageDataGenerator(rescale=1./255,rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,shear_range=0.05)

In [36]:
generator = dataGenerator.flow_from_directory(
        '/data/tam/kaggle/train_imgs/',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',shuffle=True)

def train_gen():
    while True:
        imgs, labels = next(generator)
        time.sleep(0.08)
        yield [imgs,labels], labels
train_generator = train_gen() 

Found 637965 images belonging to 2 classes.


In [17]:
test_generator = dataGenerator.flow_from_directory(
        '/data/tam/kaggle/test_imgs/',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',shuffle=True)
def val_gen():
    while True:
        imgs, labels = next(test_generator)
        yield [imgs,labels], labels
val_generator = val_gen() 

Found 3879 images belonging to 2 classes.


In [18]:
from collections import Counter

counter = Counter(generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     
from sklearn.utils import class_weight

class_weights_2 = class_weight.compute_class_weight(
               'balanced',
                np.unique(generator.classes), 
                generator.classes)

In [34]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir="./log_xception_arrcface_keras",update_freq=1000)
checkpoints = keras.callbacks.ModelCheckpoint("./log_xception_arrcface_keras/checkpoint_{epoch:04d}.pth", monitor='val_loss', verbose=0, save_best_only=False, period=3)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0)

In [ ]:
# model.fit_generator(train_generator,validation_data=val_generator,steps_per_epoch=int(637965/batch_size), epochs=100,workers=1,validation_steps=3879/batch_size,class_weight = class_weights,callbacks = [tensorboard_callback,checkpoints])
model.fit_generator(train_generator,validation_data=val_generator,steps_per_epoch=int(637965/batch_size), epochs=100,workers=1,validation_steps=3879/batch_size,class_weight = class_weights,callbacks = [tensorboard_callback,checkpoints])


Epoch 1/100
 8486/79745 [==>...........................] - ETA: 2:24:55 - loss: 12.5573 - acc: 0.7850

In [40]:
model.evaluate_generator(val_generator,3879/batch_size)


[5.422561099708126, 0.6635730858775997]